In [1]:
import os
import pickle
import warnings
import numpy as np
from util import *
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 1000
from tqdm import tqdm_notebook as tqdm

# Read Data:

In [2]:
train = pd.read_csv('train_engineered.csv')
test  = pd.read_csv('test_engineered.csv')

#############
train_labels = np.log1p(train.SalePrice)
train_ids    = train.Id
test_ids     = test.Id

train.drop(['SalePrice', 'Id'], axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)
#############

shape(train, test)

~> [train] has  1,460  rows, and  1,178  columns.
~> [test ] has  1,459  rows, and  1,178  columns.


# Read Columns:

In [3]:
ord_cols  = ['LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtExposure', 
             'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'BsmtFinType2', 'HeatingQC', 'Electrical', 
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
             'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive',
             'PoolQC', 'BedroomAbvGr', 'KitchenAbvGr']
cat_cols  = ['MSSubClass', 'MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 
            'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle', 'RoofMatl', 'Exterior1st',
            'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'GarageType', 'MiscFeature', 'SaleType', 
             'SaleCondition', 'HouseStyle', 'Fence', 'CentralAir']
num_cols  = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
            '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
            'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
date_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

In [4]:
# Load the name of the column.
with open('../3_Feature Engineering/freq_cols.pkl', 'rb') as f: 
    freq_cols = pickle.load(f)
    
with open('../3_Feature Engineering/log_cols.pkl', 'rb') as f:
    log_cols = pickle.load(f)
    
with open('../3_Feature Engineering/diff_cols.pkl', 'rb') as f:
    diff_cols = pickle.load(f)
    
with open('../3_Feature Engineering/onehot_cols.pkl', 'rb') as f:
    onehot_cols = pickle.load(f)
    
with open('../3_Feature Engineering/poly_cols.pkl', 'rb') as f:
    poly_cols = pickle.load(f)
    
with open('../3_Feature Engineering/sqrt_cols.pkl', 'rb') as f:
    sqrt_cols = pickle.load(f)
    
with open('../3_Feature Engineering/target_enc_cols.pkl', 'rb') as f:
    target_enc_cols = pickle.load(f)

with open('../3_Feature Engineering/thrmos_cols.pkl', 'rb') as f:
    thrmos_cols = pickle.load(f)

**Drop the multi-colinear features:**

In [13]:
with open('../4_Features Reduction/high_colinear_cols_to_drop.pkl', 'rb') as f:
    to_drop = pickle.load(f)
print(f'~> len of drop features: {len(to_drop)}')

~> len of drop features: 414


In [ ]:
# train.drop(to_drop, axis=1, inplace=True)
# test.drop(to_drop, axis=1, inplace=True)

# CV strategy:

In [5]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(train,
                                                      train_labels,
                                                      test_size=.2,
                                                      random_state=1)

In [64]:
scorer = make_scorer(mean_squared_error, greater_is_better=False)

def rmse_cv_train(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring=scorer, cv=10))
    return rmse

def rmse_cv_valid(model):
    rmse = np.sqrt(-cross_val_score(model, X_valid, y_valid, scoring=scorer, cv=10))
    return rmse

#Validation function
n_folds = 10

def rmsle_cv(model, X, y):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X)
    rmse = cross_validate(model, X, y, scoring="neg_mean_squared_error", cv = kf, return_train_score=True)
    return np.sqrt(-rmse['train_score']), np.sqrt(-rmse['test_score'])

# Normalization/Standardization:
Because linear models require the data to be scaled equally, I'm going to apply Normalization and Standardization and see which one works best.

In [7]:
scale_cols = target_enc_cols + sqrt_cols + poly_cols + diff_cols + log_cols + freq_cols + date_cols + ord_cols + cat_cols + num_cols

In [8]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline

In [9]:
normScaler, standScaler = MinMaxScaler(), StandardScaler()

X_train_norm = normScaler.fit_transform(train[scale_cols])
X_test_norm  = normScaler.transform(test[scale_cols])

X_train_stand = standScaler.fit_transform(train[scale_cols])
X_test_stand  = standScaler.transform(test[scale_cols])

In [10]:
X_train_norm = pd.DataFrame(X_train_norm, columns=scale_cols)
X_test_norm  = pd.DataFrame(X_test_norm, columns=scale_cols)

X_train_stand = pd.DataFrame(X_train_stand, columns=scale_cols)
X_test_stand  = pd.DataFrame(X_test_stand, columns=scale_cols)

In [11]:
X_train_norm.head()

,target_enc_means_MSSubClass,target_enc_stds_MSSubClass,target_enc_skews_MSSubClass,target_enc_medians_MSSubClass,target_enc_mins_MSSubClass,target_enc_maxs_MSSubClass,target_enc_means_MSZoning,target_enc_stds_MSZoning,target_enc_skews_MSZoning,target_enc_medians_MSZoning,target_enc_mins_MSZoning,target_enc_maxs_MSZoning,target_enc_means_Alley,target_enc_stds_Alley,target_enc_skews_Alley,target_enc_medians_Alley,target_enc_mins_Alley,target_enc_maxs_Alley,target_enc_means_LotShape,target_enc_stds_LotShape,target_enc_skews_LotShape,target_enc_medians_LotShape,target_enc_mins_LotShape,target_enc_maxs_LotShape,target_enc_means_LandContour,target_enc_stds_LandContour,target_enc_skews_LandContour,target_enc_medians_LandContour,target_enc_mins_LandContour,target_enc_maxs_LandContour,target_enc_means_LotConfig,target_enc_stds_LotConfig,target_enc_skews_LotConfig,target_enc_medians_LotConfig,target_enc_mins_LotConfig,target_enc_maxs_LotConfig,target_enc_means_Neighborhood,target_enc_stds_Neighborhood,target_enc_skews_Neighborhood,target_enc_medians_Neighborhood,target_enc_mins_Neighborhood,target_enc_maxs_Neighborhood,target_enc_means_Condition1,target_enc_stds_Condition1,target_enc_skews_Condition1,target_enc_medians_Condition1,target_enc_mins_Condition1,target_enc_maxs_Condition1,target_enc_means_Condition2,target_enc_stds_Condition2,target_enc_skews_Condition2,target_enc_medians_Condition2,target_enc_mins_Condition2,target_enc_maxs_Condition2,target_enc_means_BldgType,target_enc_stds_BldgType,target_enc_skews_BldgType,target_enc_medians_BldgType,target_enc_mins_BldgType,target_enc_maxs_BldgType,target_enc_means_RoofStyle,target_enc_stds_RoofStyle,target_enc_skews_RoofStyle,target_enc_medians_RoofStyle,target_enc_mins_RoofStyle,target_enc_maxs_RoofStyle,target_enc_means_RoofMatl,target_enc_stds_RoofMatl,target_enc_skews_RoofMatl,target_enc_medians_RoofMatl,target_enc_mins_RoofMatl,target_enc_maxs_RoofMatl,target_enc_means_Exterior1st,target_enc_stds_Exterior1st,target_enc_skews_Exterior1st,target_enc_medians_Exterior1st,target_enc_mins_Exterior1st,target_enc_maxs_Exterior1st,target_enc_means_Exterior2nd,target_enc_stds_Exterior2nd,target_enc_skews_Exterior2nd,target_enc_medians_Exterior2nd,target_enc_mins_Exterior2nd,target_enc_maxs_Exterior2nd,target_enc_means_MasVnrType,target_enc_stds_MasVnrType,target_enc_skews_MasVnrType,target_enc_medians_MasVnrType,target_enc_mins_MasVnrType,target_enc_maxs_MasVnrType,target_enc_means_Foundation,target_enc_stds_Foundation,target_enc_skews_Foundation,target_enc_medians_Foundation,target_enc_mins_Foundation,target_enc_maxs_Foundation,target_enc_means_Heating,target_enc_stds_Heating,target_enc_skews_Heating,target_enc_medians_Heating,target_enc_mins_Heating,target_enc_maxs_Heating,target_enc_means_GarageType,target_enc_stds_GarageType,target_enc_skews_GarageType,target_enc_medians_GarageType,target_enc_mins_GarageType,target_enc_maxs_GarageType,target_enc_means_MiscFeature,target_enc_stds_MiscFeature,target_enc_skews_MiscFeature,target_enc_medians_MiscFeature,target_enc_mins_MiscFeature,target_enc_maxs_MiscFeature,target_enc_means_SaleType,target_enc_stds_SaleType,target_enc_skews_SaleType,target_enc_medians_SaleType,target_enc_mins_SaleType,target_enc_maxs_SaleType,target_enc_means_SaleCondition,target_enc_stds_SaleCondition,target_enc_skews_SaleCondition,target_enc_medians_SaleCondition,target_enc_mins_SaleCondition,target_enc_maxs_SaleCondition,target_enc_means_HouseStyle,target_enc_stds_HouseStyle,target_enc_skews_HouseStyle,target_enc_medians_HouseStyle,target_enc_mins_HouseStyle,target_enc_maxs_HouseStyle,target_enc_means_Fence,target_enc_stds_Fence,target_enc_skews_Fence,target_enc_medians_Fence,target_enc_mins_Fence,target_enc_maxs_Fence,target_enc_means_CentralAir,target_enc_stds_CentralAir,target_enc_skews_CentralAir,target_enc_medians_CentralAir,target_enc_mins_CentralAir,target_enc_maxs_CentralAir,target_enc_means_LandSlope,target_enc_stds_LandSlope,target_enc_skews_LandSlope,target_

# Modeling:
## Normalized Data:
### Thermometer [Ordinal] + OneHot Encoding [Nominal]: LB(10)

In [33]:
from sklearn.linear_model import LinearRegression

def submit_csv(model, test, name):
    submit = pd.read_csv('../0_Data/sample_submission.csv')
    submit['SalePrice'] = model.predict(test)
    submit.to_csv(f'{name}.csv', index=False)
    
def LRModel(train, test, name):
    lr = LinearRegression()
    lr.fit(train, train_labels)
    train_score, test_score = rmsle_cv(lr, train, (train_labels))

    print(f'Linear Regression Score [Train]: {train_score.mean()}')
    print(f'Linear Regression Score [Valid]: {test_score.mean()}')
    
    submit_csv(lr, test, f'./output/'+name)

In [39]:
lr = LRModel(train[thrmos_cols+onehot_cols], test[thrmos_cols+onehot_cols], '0_lr_1')

Linear Regression Score [Train]: 0.11285897035220545
Linear Regression Score [Valid]: 18402405325.586407


### Previous + Target Encoding: LB(7.34513)

In [34]:
train_temp = pd.concat([train[thrmos_cols+onehot_cols], X_train_norm[target_enc_cols]], axis=1)
test_temp  = pd.concat([test[thrmos_cols+onehot_cols], X_test_norm[target_enc_cols]], axis=1)

In [35]:
LRModel(train_temp, test_temp, '1_lr_2')

Linear Regression Score [Train]: 0.10699790562566276
Linear Regression Score [Valid]: 27465057512.897552


### Previous + Frequency Encoding:

In [36]:
train_temp = pd.concat([train[thrmos_cols+onehot_cols], X_train_norm[target_enc_cols+freq_cols]], axis=1)
test_temp  = pd.concat([test[thrmos_cols+onehot_cols], X_test_norm[target_enc_cols+freq_cols]], axis=1)

In [37]:
LRModel(train_temp, test_temp, '2_lr_3')

Linear Regression Score [Train]: 0.10705751009440274
Linear Regression Score [Valid]: 46477553198.17616


### onehot_cols + thrmos_cols + target_enc_cols + num_cols: LB(12)

In [40]:
train_temp = pd.concat([train[thrmos_cols+onehot_cols], X_train_norm[target_enc_cols+num_cols]], axis=1)
test_temp  = pd.concat([test[thrmos_cols+onehot_cols], X_test_norm[target_enc_cols+num_cols]], axis=1)

In [41]:
LRModel(train_temp, test_temp, '2_lr')

Linear Regression Score [Train]: 0.0953640731433418
Linear Regression Score [Valid]: 32975186430.37146


### onehot_cols + thrmos_cols + target_enc_cols + log_cols + sqrt_cols: LB(8.02617)

In [42]:
train_temp = pd.concat([train[thrmos_cols+onehot_cols], X_train_norm[target_enc_cols+log_cols+sqrt_cols]], axis=1)
test_temp  = pd.concat([test[thrmos_cols+onehot_cols], X_test_norm[target_enc_cols+log_cols+sqrt_cols]], axis=1)

In [43]:
LRModel(train_temp, test_temp, '3_lr')

Linear Regression Score [Train]: 0.09047830460961834
Linear Regression Score [Valid]: 34516736419.69169


### all data: LB(11.95685)

In [44]:
train_temp = pd.concat([train[thrmos_cols+onehot_cols], X_train_norm], axis=1)
test_temp  = pd.concat([test[thrmos_cols+onehot_cols], X_test_norm], axis=1)

In [45]:
LRModel(train_temp, test_temp, '4_lr')

Linear Regression Score [Train]: 0.070418664165666
Linear Regression Score [Valid]: 14421962581.724142


### After dropping highly correlated features: LB(8.07127)

In [66]:
train_temp = pd.concat([train[onehot_cols+thrmos_cols], X_train_norm], axis=1).drop(to_drop, axis=1)
test_temp  = pd.concat([test[onehot_cols+thrmos_cols], X_test_norm], axis=1).drop(to_drop, axis=1)

In [67]:
LRModel(train_temp, test_temp, '5_lr')

Linear Regression Score [Train]: 0.07433064110226208
Linear Regression Score [Valid]: 52269848210.185524
